In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('../../../module/')

from keras2.models import Model
from keras2.layers import concatenate, Dense, Input, Flatten
from keras2.optimizers import Adam
import csv
from util import *
import gym2
from rl2.agents import selfDDPGAgent, selfDDPGAgent2
from rl2.memory import SequentialMemory

Using TensorFlow backend.
Using TensorFlow backend.


In [3]:
env = gym2.make('Linear-v0')

In [4]:
def critic_net(a_shape , s_shape):
    action_input = Input(a_shape)
    observation_input = Input(shape=(1,)+s_shape)
    flattened_observation = Flatten()(observation_input)
    x = concatenate([action_input, flattened_observation])
    x = Dense(16, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    critic = Model(inputs=[action_input, observation_input], outputs=x)
    return (critic, action_input)

def branch_actor(a_shape, s_shape):
    action_input = Input(shape=(1,)+s_shape)
    x = Flatten()(action_input) # 実質的なinput layer
    
    x1 = Dense(8, activation="relu")(x)
    x1 = Dense(8, activation="relu")(x1)
    x1 = Dense(1, activation="multiple_tanh")(x1) # action signal
    
    x2 = Dense(8, activation="relu")(x)
    x2 = Dense(8, activation="relu")(x2)
    x2 = Dense(1, activation="tau_output")(x2) # tau
    
    output = concatenate([x1, x2])
    actor = Model(inputs=action_input, outputs=output)
    return actor


def agent2(a_shape, s_shape):
    actor = branch_actor(a_shape, s_shape)
    critic, critic_action_input = critic_net(a_shape, s_shape)
    memory = SequentialMemory(limit = 50000, window_length = 1)
    agent = selfDDPGAgent2(
        a_shape[0],
        actor,
        critic,
        critic_action_input,
        memory,
        mb_noise=False,
        coef_u = 2.,
        coef_tau = .5,
        action_clipper=[-10., 10.],
        tau_clipper=[0.001, 1.],
        params_logging=False,
        gradient_logging=False,
        batch_size=128,
    )
    return agent

In [5]:
#learning   
l = .1
step = 1000000  # num of interval
episode_step = step
a = agent2((2,), (2,))
actor_optimizer, critic_optimizer = Adam(lr=100., clipnorm=1.), Adam(lr=0.001, clipnorm=1.) # actorの方は何でもいい
optimizer = [actor_optimizer, critic_optimizer]
a.compile(optimizer=optimizer, metrics=["mse"], action_lr=0.0001, tau_lr=0.001)

In [ ]:
a.load_weights('../saved_agent/linear_init.h5')
out = a.fit(env, l=l, nb_steps=step, visualize=0, verbose=1, nb_max_episode_steps=episode_step, episode_time=5.)

Training for 1000000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 118s 12ms/step - reward: -0.0056
26 episodes - episode_reward: -2.047 [-6.113, 0.443] - loss: 0.001 - mean_squared_error: 0.003 - mean_q: 0.421

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 290s 29ms/step - reward: -0.0089
22 episodes - episode_reward: -4.089 [-5.024, -3.178] - loss: 0.003 - mean_squared_error: 0.005 - mean_q: 1.076

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 358s 36ms/step - reward: -0.0097
22 episodes - episode_reward: -4.428 [-4.713, -3.951] - loss: 0.003 - mean_squared_error: 0.005 - mean_q: 1.175

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 245s 24ms/step - reward: -0.0094
22 episodes - episode_reward: -4.244 [-4.631, -3.826] - loss: 0.002 - mean_squared_error: 0.004 - mean_q: 1.084

Interval 5 (40000 steps performed)
10000/10000 [==============